In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
from darts.dataprocessing import Pipeline

from tqdm import tqdm

import metaestimators
#from darts.models import LightGBMModel




ModuleNotFoundError: No module named 'metaestimators'

# Import des données 

In [65]:
train = pd.read_csv("data/modifie/trainFinal.csv")
display(train)

test = pd.read_csv("data/modifie/testFinal.csv")
display(test)

date  store_nbr                      family     sales  \
0        2013-01-01          1                  AUTOMOTIVE     0.000   
1        2013-01-01          1                   BABY CARE     0.000   
2        2013-01-01          1                      BEAUTY     0.000   
3        2013-01-01          1                   BEVERAGES     0.000   
4        2013-01-01          1                       BOOKS     0.000   
...             ...        ...                         ...       ...   
3000883  2017-08-15          9                     POULTRY   438.133   
3000884  2017-08-15          9              PREPARED FOODS   154.553   
3000885  2017-08-15          9                     PRODUCE  2419.729   
3000886  2017-08-15          9  SCHOOL AND OFFICE SUPPLIES   121.000   
3000887  2017-08-15          9                     SEAFOOD    16.000   

         onpromotion   city      state  flagHoliday  transactions  dcoilwtico  \
0                  0  Quito  Pichincha            1           NaN       93.12   
1                  0  Quito  Pichincha            1           NaN       93.12   
2                  0  Quito  Pichincha            1           NaN       93.12   
3                  0  Quito  Pichincha            1           NaN       93.12   
4                  0  Quito  Pichincha            1           NaN       93.12   
...              ...    ...        ...          ...           ...         ...   
3000883            0  Quito  Pichincha            0        2155.0       47.57   
3000884            1  Quito  Pichincha            0        2155.0       47.57   
3000885          148  Quito  Pichincha            0        2155.0       47.57   
3000886            8  Quito  Pichincha            0        2155.0       47.57   
3000887            0  Quito  Pichincha            0        2155.0       47.57   

        type  cluster  jour  mois  annee  jourSemaine  jourAnnee  semaineAnnee  
0          D       13     1     1   2013            1          1             1  
1          D       13     1     1   2013            1          1             1  
2          D       13     1     1   2013            1          1             1  
3          D       13     1     1   2013            1          1             1  
4          D       13     1     1   2013            1          1             1  
...      ...      ...   ...   ...    ...          ...        ...           ...  
3000883    B        6    15     8   2017            1        227            33  
3000884    B        6    15     8   2017            1        227            33  
3000885    B        6    15     8   2017            1        227            33  
3000886    B        6    15     8   2017            1        227            33  
3000887    B        6    15     8   2017            1        227            33  

[3000888 rows x 18 columns]

date  store_nbr                      family  onpromotion  sales  \
0      2017-08-16          1                  AUTOMOTIVE            0    NaN   
1      2017-08-16          1                   BABY CARE            0    NaN   
2      2017-08-16          1                      BEAUTY            2    NaN   
3      2017-08-16          1                   BEVERAGES           20    NaN   
4      2017-08-16          1                       BOOKS            0    NaN   
...           ...        ...                         ...          ...    ...   
28507  2017-08-31          9                     POULTRY            1    NaN   
28508  2017-08-31          9              PREPARED FOODS            0    NaN   
28509  2017-08-31          9                     PRODUCE            1    NaN   
28510  2017-08-31          9  SCHOOL AND OFFICE SUPPLIES            9    NaN   
28511  2017-08-31          9                     SEAFOOD            0    NaN   

        city      state  flagHoliday  transactions  dcoilwtico type  cluster  \
0      Quito  Pichincha            0           NaN       46.80    D       13   
1      Quito  Pichincha            0           NaN       46.80    D       13   
2      Quito  Pichincha            0           NaN       46.80    D       13   
3      Quito  Pichincha            0           NaN       46.80    D       13   
4      Quito  Pichincha            0           NaN       46.80    D       13   
...      ...        ...          ...           ...         ...  ...      ...   
28507  Quito  Pichincha            0           NaN       47.26    B        6   
28508  Quito  Pichincha            0           NaN       47.26    B        6   
28509  Quito  Pichincha            0           NaN       47.26    B        6   
28510  Quito  Pichincha            0           NaN       47.26    B        6   
28511  Quito  Pichincha            0           NaN       47.26    B        6   

       jour  mois  annee  jourSemaine  jourAnnee  semaineAnnee  
0        16     8   2017            2        228            33  
1        16     8   2017            2        228            33  
2        16     8   2017            2        228            33  
3        16     8   2017            2        228            33  
4        16     8   2017            2        228            33  
...     ...   ...    ...          ...        ...           ...  
28507    31     8   2017            3        243            35  
28508    31     8   2017            3        243            35  
28509    31     8   2017            3        243            35  
28510    31     8   2017            3        243            35  
28511    31     8   2017            3        243            35  

[28512 rows x 18 columns]

In [71]:
col = ['family','city','state','type','jour','mois','annee','jourSemaine','jourAnnee','semaineAnnee']
for c in col :
    test['%sCat' % c] = test[c].astype('category').cat.codes
    train['%sCat' % c] = train[c].astype('category').cat.codes

test = test.drop(col, axis=1)    
train = train.drop(col, axis=1)
display(test)
display(train)



date  store_nbr  onpromotion  sales  flagHoliday  transactions  \
0      2017-08-16          1            0    NaN            0           NaN   
1      2017-08-16          1            0    NaN            0           NaN   
2      2017-08-16          1            2    NaN            0           NaN   
3      2017-08-16          1           20    NaN            0           NaN   
4      2017-08-16          1            0    NaN            0           NaN   
...           ...        ...          ...    ...          ...           ...   
28507  2017-08-31          9            1    NaN            0           NaN   
28508  2017-08-31          9            0    NaN            0           NaN   
28509  2017-08-31          9            1    NaN            0           NaN   
28510  2017-08-31          9            9    NaN            0           NaN   
28511  2017-08-31          9            0    NaN            0           NaN   

       dcoilwtico  cluster  familyCat  cityCat  stateCat  typeCat  jourCat  \
0           46.80       13          0       18        12        3        0   
1           46.80       13          1       18        12        3        0   
2           46.80       13          2       18        12        3        0   
3           46.80       13          3       18        12        3        0   
4           46.80       13          4       18        12        3        0   
...           ...      ...        ...      ...       ...      ...      ...   
28507       47.26        6         28       18        12        1       15   
28508       47.26        6         29       18        12        1       15   
28509       47.26        6         30       18        12        1       15   
28510       47.26        6         31       18        12        1       15   
28511       47.26        6         32       18        12        1       15   

       moisCat  anneeCat  jourSemaineCat  jourAnneeCat  semaineAnneeCat  
0            0         0               2             0                0  
1            0         0               2             0                0  
2            0         0               2             0                0  
3            0         0               2             0                0  
4            0         0               2             0                0  
...        ...       ...             ...           ...              ...  
28507        0         0               3            15                2  
28508        0         0               3            15                2  
28509        0         0               3            15                2  
28510        0         0               3            15                2  
28511        0         0               3            15                2  

[28512 rows x 18 columns]

date  store_nbr     sales  onpromotion  flagHoliday  \
0        2013-01-01          1     0.000            0            1   
1        2013-01-01          1     0.000            0            1   
2        2013-01-01          1     0.000            0            1   
3        2013-01-01          1     0.000            0            1   
4        2013-01-01          1     0.000            0            1   
...             ...        ...       ...          ...          ...   
3000883  2017-08-15          9   438.133            0            0   
3000884  2017-08-15          9   154.553            1            0   
3000885  2017-08-15          9  2419.729          148            0   
3000886  2017-08-15          9   121.000            8            0   
3000887  2017-08-15          9    16.000            0            0   

         transactions  dcoilwtico  cluster  familyCat  cityCat  stateCat  \
0                 NaN       93.12       13          0       18        12   
1                 NaN       93.12       13          1       18        12   
2                 NaN       93.12       13          2       18        12   
3                 NaN       93.12       13          3       18        12   
4                 NaN       93.12       13          4       18        12   
...               ...         ...      ...        ...      ...       ...   
3000883        2155.0       47.57        6         28       18        12   
3000884        2155.0       47.57        6         29       18        12   
3000885        2155.0       47.57        6         30       18        12   
3000886        2155.0       47.57        6         31       18        12   
3000887        2155.0       47.57        6         32       18        12   

         typeCat  jourCat  moisCat  anneeCat  jourSemaineCat  jourAnneeCat  \
0              3        0        0         0               1             0   
1              3        0        0         0               1             0   
2              3        0        0         0               1             0   
3              3        0        0         0               1             0   
4              3        0        0         0               1             0   
...          ...      ...      ...       ...             ...           ...   
3000883        1       14        7         4               1           226   
3000884        1       14        7         4               1           226   
3000885        1       14        7         4               1           226   
3000886        1       14        7         4               1           226   
3000887        1       14        7         4               1           226   

         semaineAnneeCat  
0                      0  
1                      0  
2                      0  
3                      0  
4                      0  
...                  ...  
3000883               32  
3000884               32  
3000885               32  
3000886               32  
3000887               32  

[3000888 rows x 18 columns]

In [72]:
df = pd.concat([train,test]).drop_duplicates()


## On groupe nos données selon la famille produit et le magasin

In [75]:
display(df)

date  store_nbr  sales  onpromotion  flagHoliday  transactions  \
0      2013-01-01          1    0.0            0            1           NaN   
1      2013-01-01          1    0.0            0            1           NaN   
2      2013-01-01          1    0.0            0            1           NaN   
3      2013-01-01          1    0.0            0            1           NaN   
4      2013-01-01          1    0.0            0            1           NaN   
...           ...        ...    ...          ...          ...           ...   
28507  2017-08-31          9    NaN            1            0           NaN   
28508  2017-08-31          9    NaN            0            0           NaN   
28509  2017-08-31          9    NaN            1            0           NaN   
28510  2017-08-31          9    NaN            9            0           NaN   
28511  2017-08-31          9    NaN            0            0           NaN   

       dcoilwtico  cluster  familyCat  cityCat  stateCat  typeCat  jourCat  \
0           93.12       13          0       18        12        3        0   
1           93.12       13          1       18        12        3        0   
2           93.12       13          2       18        12        3        0   
3           93.12       13          3       18        12        3        0   
4           93.12       13          4       18        12        3        0   
...           ...      ...        ...      ...       ...      ...      ...   
28507       47.26        6         28       18        12        1       15   
28508       47.26        6         29       18        12        1       15   
28509       47.26        6         30       18        12        1       15   
28510       47.26        6         31       18        12        1       15   
28511       47.26        6         32       18        12        1       15   

       moisCat  anneeCat  jourSemaineCat  jourAnneeCat  semaineAnneeCat  
0            0         0               1             0                0  
1            0         0               1             0                0  
2            0         0               1             0                0  
3            0         0               1             0                0  
4            0         0               1             0                0  
...        ...       ...             ...           ...              ...  
28507        0         0               3            15                2  
28508        0         0               3            15                2  
28509        0         0               3            15                2  
28510        0         0               3            15                2  
28511        0         0               3            15                2  

[3029400 rows x 18 columns]

In [70]:
familyCatListe = df.family.unique()

familyDico = {}

for family in tqdm(familyCatListe):
    dfFam = df.loc[df['family'] == family]
    #    dfFam = df.loc[df['familyCat'] == family]


    listeFamilyDico = TimeSeries.from_group_dataframe(
                                dfFam,
                                time_col="date",
                                group_cols=["store_nbr","family"], # columns for grouping time series
        #                                group_cols=["store_nbr","familyCat"], # columns for grouping time series
#                                static_cols=['onpromotion','flagHoliday','transactions','dcoilwtico','cluster','cityCat','stateCat','typeCat','jourCat','moisCat','anneeCat','jourSemaineCat','jourAnneeCat','semaineAnneeCat'], # variables explicatives (covariates)

                                static_cols=['onpromotion','flagHoliday','transactions','dcoilwtico','cluster','city','state','type','jour','mois','annee','jourSemaine','jourAnnee','semaineAnnee'], # variables explicatives (covariates)
                                value_cols="sales", # variable expliquée (target),
                                fill_missing_dates=True, # filling missing dates, remember Dec 25th
                                freq='D' # jour
                                )
    for ts in listeFamilyDico:
            ts = ts.astype(np.float32)

    listeFamilyDico = sorted(familyDico, key=lambda ts: int(ts.static_covariates_values()[0,0]))
    familyDico[family] = listeFamilyDico

  0%|          | 0/33 [00:00<?, ?it/s]ValueError: Encountered more than one unique value in group             store_nbr      family  onpromotion  flagHoliday  transactions  \
date                                                                        
2013-01-01          1  AUTOMOTIVE            0            1           NaN   
2013-01-02          1  AUTOMOTIVE            0            0        2111.0   
2013-01-03          1  AUTOMOTIVE            0            0        1833.0   
2013-01-04          1  AUTOMOTIVE            0            0        1863.0   
2013-01-05          1  AUTOMOTIVE            0            1        1509.0   
...               ...         ...          ...          ...           ...   
2017-08-27          1  AUTOMOTIVE            0            0           NaN   
2017-08-28          1  AUTOMOTIVE            0            0           NaN   
2017-08-29          1  AUTOMOTIVE            0            0           NaN   
2017-08-30          1  AUTOMOTIVE            0         

ValueError: Encountered more than one unique value in group             store_nbr      family  onpromotion  flagHoliday  transactions  \
date                                                                        
2013-01-01          1  AUTOMOTIVE            0            1           NaN   
2013-01-02          1  AUTOMOTIVE            0            0        2111.0   
2013-01-03          1  AUTOMOTIVE            0            0        1833.0   
2013-01-04          1  AUTOMOTIVE            0            0        1863.0   
2013-01-05          1  AUTOMOTIVE            0            1        1509.0   
...               ...         ...          ...          ...           ...   
2017-08-27          1  AUTOMOTIVE            0            0           NaN   
2017-08-28          1  AUTOMOTIVE            0            0           NaN   
2017-08-29          1  AUTOMOTIVE            0            0           NaN   
2017-08-30          1  AUTOMOTIVE            0            0           NaN   
2017-08-31      

# Normalisation

## ToDo : séparer le dataFrame par magasin/famille et Pipeline([train_filler, static_cov_transformer, log_transformer, train_scaler]

In [81]:
scaler = MinMaxScaler()
df = df.set_index('date')
for col in df.columns:
    df['%sSc' % col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
    df = df.drop(col, axis=1)
display(df)

onpromotionSc  flagHolidaySc  transactionsSc  dcoilwticoSc  \
date                                                                     
2013-01-01       0.000000            1.0             NaN      0.792728   
2013-01-01       0.000000            1.0             NaN      0.792728   
2013-01-01       0.000000            1.0             NaN      0.792728   
2013-01-01       0.000000            1.0             NaN      0.792728   
2013-01-01       0.000000            1.0             NaN      0.792728   
...                   ...            ...             ...           ...   
2017-08-31       0.001350            0.0             NaN      0.249556   
2017-08-31       0.000000            0.0             NaN      0.249556   
2017-08-31       0.001350            0.0             NaN      0.249556   
2017-08-31       0.012146            0.0             NaN      0.249556   
2017-08-31       0.000000            0.0             NaN      0.249556   

            clusterSc  familyCatSc  cityCatSc  stateCatSc  typeCatSc  \
date                                                                   
2013-01-01     0.7500      0.00000   0.857143         0.8       0.75   
2013-01-01     0.7500      0.03125   0.857143         0.8       0.75   
2013-01-01     0.7500      0.06250   0.857143         0.8       0.75   
2013-01-01     0.7500      0.09375   0.857143         0.8       0.75   
2013-01-01     0.7500      0.12500   0.857143         0.8       0.75   
...               ...          ...        ...         ...        ...   
2017-08-31     0.3125      0.87500   0.857143         0.8       0.25   
2017-08-31     0.3125      0.90625   0.857143         0.8       0.25   
2017-08-31     0.3125      0.93750   0.857143         0.8       0.25   
2017-08-31     0.3125      0.96875   0.857143         0.8       0.25   
2017-08-31     0.3125      1.00000   0.857143         0.8       0.25   

            jourCatSc  moisCatSc  anneeCatSc  jourSemaineCatSc  \
date                                                             
2013-01-01        0.0        0.0         0.0          0.166667   
2013-01-01        0.0        0.0         0.0          0.166667   
2013-01-01        0.0        0.0         0.0          0.166667   
2013-01-01        0.0        0.0         0.0          0.166667   
2013-01-01        0.0        0.0         0.0          0.166667   
...               ...        ...         ...               ...   
2017-08-31        0.5        0.0         0.0          0.500000   
2017-08-31        0.5        0.0         0.0          0.500000   
2017-08-31        0.5        0.0         0.0          0.500000   
2017-08-31        0.5        0.0         0.0          0.500000   
2017-08-31        0.5        0.0         0.0          0.500000   

            jourAnneeCatSc  semaineAnneeCatSc  dateScSc  store_nbrScSc  
date                                                                    
2013-01-01        0.000000           0.000000       0.0       0.000000  
2013-01-01        0.000000           0.000000       0.0       0.000000  
2013-01-01        0.000000           0.000000       0.0       0.000000  
2013-01-01        0.000000           0.000000       0.0       0.000000  
2013-01-01        0.000000           0.000000       0.0       0.000000  
...                    ...                ...       ...            ...  
2017-08-31        0.041096           0.038462       NaN       0.150943  
2017-08-31        0.041096           0.038462       NaN       0.150943  
2017-08-31        0.041096           0.038462       NaN       0.150943  
2017-08-31        0.041096           0.038462       NaN       0.150943  
2017-08-31        0.041096           0.038462       NaN       0.150943  

[3029400 rows x 17 columns]

In [76]:
t = df.copy()
display(t)

date  store_nbr  sales  onpromotion  flagHoliday  transactions  \
0      2013-01-01          1    0.0            0            1           NaN   
1      2013-01-01          1    0.0            0            1           NaN   
2      2013-01-01          1    0.0            0            1           NaN   
3      2013-01-01          1    0.0            0            1           NaN   
4      2013-01-01          1    0.0            0            1           NaN   
...           ...        ...    ...          ...          ...           ...   
28507  2017-08-31          9    NaN            1            0           NaN   
28508  2017-08-31          9    NaN            0            0           NaN   
28509  2017-08-31          9    NaN            1            0           NaN   
28510  2017-08-31          9    NaN            9            0           NaN   
28511  2017-08-31          9    NaN            0            0           NaN   

       dcoilwtico  cluster  familyCat  cityCat  stateCat  typeCat  jourCat  \
0           93.12       13          0       18        12        3        0   
1           93.12       13          1       18        12        3        0   
2           93.12       13          2       18        12        3        0   
3           93.12       13          3       18        12        3        0   
4           93.12       13          4       18        12        3        0   
...           ...      ...        ...      ...       ...      ...      ...   
28507       47.26        6         28       18        12        1       15   
28508       47.26        6         29       18        12        1       15   
28509       47.26        6         30       18        12        1       15   
28510       47.26        6         31       18        12        1       15   
28511       47.26        6         32       18        12        1       15   

       moisCat  anneeCat  jourSemaineCat  jourAnneeCat  semaineAnneeCat  
0            0         0               1             0                0  
1            0         0               1             0                0  
2            0         0               1             0                0  
3            0         0               1             0                0  
4            0         0               1             0                0  
...        ...       ...             ...           ...              ...  
28507        0         0               3            15                2  
28508        0         0               3            15                2  
28509        0         0               3            15                2  
28510        0         0               3            15                2  
28511        0         0               3            15                2  

[3029400 rows x 18 columns]